In [1]:
import boto3
import json
from botocore.exceptions import ClientError
from botocore.exceptions import EndpointConnectionError
import time

region_aws = 'us-east-1'

# Create session
session = boto3.Session(profile_name='badbob')
ec2_client = session.client('ec2', region_name=region_aws)
ec2_resource = session.resource('ec2', region_name=region_aws)
kms_client = session.client('kms', region_name=region_aws)
autoscaling_client = session.client('autoscaling')

In [50]:
instance_id = 'i-053acd466af88d6ac'
kms_key = 'alias/aws/ebs'

### 0) Check the instance

In [53]:
try:
    # Is the instance is in the good state (or exists) ?
    # If not, raise exception
    ec2_client.describe_instances(InstanceIds=[instance_id])
    instance = ec2_resource.Instance(id=instance_id)
    print(f'Instance {instance_id} exists')
    
except (EndpointConnectionError, ValueError) as error:
    print(f'Problem with your AWS region ? {error}')

except (ClientError, TypeError) as error:
    print(f'Problem with the instance {error}')

else:
    # Check if the instance are attached to an Auto Scaling group:
    response = autoscaling_client.describe_auto_scaling_instances(InstanceIds=[instance_id])
    if len(response['AutoScalingInstances']) > 0:
        asg_name = response['AutoScalingInstances'][0]['AutoScalingGroupName']
        print("\n-- Instance is attached to an Auto Scaling group --")
        print("If you stop the instances, Amazon EC2 Auto Scaling might launch replacement instances automatically. \
            \nIf you do not want Amazon EC2 Auto Scaling to launch replacement instances, first detach the instances \
            \nfrom the Auto Scaling group.")
        print(f'\nAuto Scaling group name: {asg_name}\n')
        print("Try to suspend the Auto Scaling processes: https://docs.aws.amazon.com/autoscaling/ec2/userguide/get-started-with-ec2-auto-scaling.html?icmpid=docs_ec2as_help_panel")

Instance i-053acd466af88d6ac exists

-- Instance is attached to an Auto Scaling group --
If you stop the instances, Amazon EC2 Auto Scaling might launch replacement instances automatically.             
If you do not want Amazon EC2 Auto Scaling to launch replacement instances, first detach the instances             
from the Auto Scaling group.

Auto Scaling group name: cfn-ECSAutoScalingGroup-IX1LXS9NH2F2

Try to suspend the Auto Scaling processes: https://docs.aws.amazon.com/autoscaling/ec2/userguide/get-started-with-ec2-auto-scaling.html?icmpid=docs_ec2as_help_panel


### 1) Create Snapshot

In [41]:
# Get the root volume ID of the EBS volume attached to the EC2 instance
for device in instance.block_device_mappings:
    if device['DeviceName']==instance.root_device_name:
        volume_id = device['Ebs']['VolumeId']
        print("The root volume is", volume_id)
    else:
        print("The additional ebs volume is", device['Ebs']['VolumeId'])

volume = ec2_resource.Volume(volume_id)
volume.create_tags(Tags=[{'Key': 'volume_source', 'Value': volume_id}])
snapshot = volume.create_snapshot(Description=f'Snapshot of {volume_id}', TagSpecifications=[{'ResourceType': 'snapshot', 'Tags': volume.tags}])
snapshot.create_tags(Tags=[{'Key': 'volume_source', 'Value': volume_id}])
snapshot_id = snapshot.id

snapshot.create_tags(Tags=[{'Key': 'Name', 'Value': f'SNAP - {volume.id}'}])

print("Wait for the snapshot to be completed...")
while snapshot.state != 'completed':
    time.sleep(10)
    snapshot.reload()

The root volume is vol-0a563dd467ab6bc94
Wait for the snapshot to be completed...


### 2) Encrypt the Snapshot

In [42]:
if snapshot.encrypted == True:
    print("Snapshot is already encrypted go to next step")
else:
    # Encrypt the snapshot
    print("Encrypting the snapshot")
    response = ec2_client.copy_snapshot(
        SourceRegion=region_aws,
        SourceSnapshotId=snapshot_id,
        Description='Encrypted copy of snapshot ' + snapshot_id,
        Encrypted=True,
        KmsKeyId=kms_key,
        TagSpecifications=[
            {
                'ResourceType': 'snapshot',
                'Tags': snapshot.tags
            }
        ]
    )
    encrypted_snapshot_id = response['SnapshotId']
    snapshot_encypted = ec2_resource.Snapshot(encrypted_snapshot_id)

snapshot_encypted.create_tags(Tags=[{'Key': 'Name', 'Value': f'SNAP - encrypt - {volume.id}'}])

print("Wait for the encrypted snapshot to be completed...")
while snapshot_encypted.state != 'completed':
    time.sleep(20)
    snapshot_encypted.reload()

Encrypting the snapshot
Wait for the encrypted snapshot to be completed...


### 3) Create volume encrypted from snapshot

In [43]:
volume_type = volume.volume_type
az = volume.availability_zone

volume_encrypted = ec2_resource.create_volume(
        SnapshotId=encrypted_snapshot_id,
        VolumeType=volume_type,
        TagSpecifications=[
            {
                'ResourceType': 'volume',
                'Tags': snapshot_encypted.tags
            }
        ],
        AvailabilityZone = az
)

volume_encrypted.create_tags(Tags=[{'Key': 'Name', 'Value': f'EBS encrypt -{volume.id}'}])

[ec2.Tag(resource_id='vol-083ca0ea898b8aea5', key='Name', value='EBS encrypt -vol-0a563dd467ab6bc94')]

### 4) Stop instance!

In [54]:
if instance.state['Name'] != 'stopped':
    # Stop instance
    response = ec2_client.stop_instances(InstanceIds=[instance_id])
    print(f'Stopping instance {instance_id} ...')
else:
    print(f'Instance {instance_id} already stopped !')
            
print("Wait for the encrypted snapshot to be available...")
while volume_encrypted.state != 'available':
    time.sleep(10)
    volume_encrypted.reload()

print("Wait for stop instance...")
while instance.state['Name'] != 'stopped':
    time.sleep(10)
    instance.reload()

Stopping instance i-053acd466af88d6ac ...
Wait for the encrypted snapshot to be available...
Wait for stop instance...


### 4) Unmount the EBS volume

In [55]:
# Extract device name (ex: /dev/xvda) and DeleteOnTermination flag
device = volume.attachments[0]['Device']
flag = volume.attachments[0]['DeleteOnTermination']

try:
    response_deatch = volume.detach_from_instance(InstanceId=instance_id, Device=device)
    time.sleep(5)
    response_attach = instance.attach_volume(Device=device, VolumeId=volume_encrypted.id)
    time.sleep(5)
except Exception as e:
    print(f"Error: {e}")
else:
    print(f"Volume {volume.id} detached from {instance_id} and attached {volume_encrypted.id}")

Volume vol-0dec0c136de581baa detached from i-053acd466af88d6ac and attached vol-083ca0ea898b8aea5


### 5) Start instance

In [56]:
print("Wait until the encrypted volume is in use...")
while volume_encrypted.state != 'in-use':
    time.sleep(5)
    volume_encrypted.reload()

print(f"Start the instance {instance.id}...")
instance.start()

Wait until the encrypted volume is in use...
Start the instance i-053acd466af88d6ac...


{'StartingInstances': [{'CurrentState': {'Code': 0, 'Name': 'pending'},
   'InstanceId': 'i-053acd466af88d6ac',
   'PreviousState': {'Code': 80, 'Name': 'stopped'}}],
 'ResponseMetadata': {'RequestId': 'e1e2c6f5-d466-4ae5-8b63-b6d0df70db80',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e1e2c6f5-d466-4ae5-8b63-b6d0df70db80',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '579',
   'date': 'Fri, 28 Jul 2023 20:54:39 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

### 6) Clean up

#### Delete the volume unencrypted

In [57]:
print(f"Deleting unencrypted volume {volume.id}...  ")
response_vol = volume.delete()

Deleting unencrypted volume vol-0dec0c136de581baa...  


#### Delete snapshots

In [58]:
print(f"Deleting unencrypted snapshot {snapshot.id}...  ")
response_snap = snapshot.delete()

print(f"Deleting encrypted snapshot {snapshot_encypted.id}...  ")
response_snap_encrypted = snapshot_encypted.delete()

Deleting unencrypted snapshot snap-0bd6e7de553cfd6c3...  
Deleting encrypted snapshot snap-0e0d85c69676ef1ef...  
